In [1]:
import spacy
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
import numpy as np

In [2]:
def readArticle(input_file_path):
    """
        file path: where the input file located 
        
        return: a dataframe
    """
    file = open(input_file_path, 'r', encoding = 'utf8')
    articles_dt = file.read().split('\n')[:-1]
    pubId, canonicalUrl,firstScrape,title,text,lang_reliability = [],[],[],[],[],[]
    for article in articles_dt:    
        row = article.split('\t')
        pubId.append(row[0])
        canonicalUrl.append(row[4])
        firstScrape.append(row[5])
        lang_reliability.append(row[7])
        title.append(row[8])
        text.append(row[9])
    articles_df = pd.DataFrame()
    articles_df['pubId'], articles_df['canonicalUrl'], articles_df['firstScrape'], articles_df['title'], articles_df['text'], articles_df['lang_reliability']= pubId, canonicalUrl,firstScrape,title,text,lang_reliability
    return articles_df

In [3]:
## text normzalization
STOPWORDS = set(stopwords.words('english'))
def clean_text(text):
    """
        text: a article 
        
        return: normalized the text data
    """
    text = re.sub(r"[^\w\s]", '', text) ## Remove all non-word characters (everything except numbers and letters)
    text = text.lower() # lowercase text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    #text = lemmatize_sentence(text)
    return text

In [6]:
# from the Naming clusters we realize some lemmalization issue. 
# in this function we will cusomized combine those words together
customized_dict = {"russian":'russia',"russians":"russia","white house":"the white house", "us":"the united states", 
                   "american":"the united states", "america":"the united states", "republicans":"republican",
                   "democrats":"democrat", "democratic":"democrat","trump":"donald trump", "trumps":"donald trump", 
                  "putin":"vladimir putin"}
def cusomizedNameEntity(word):
    if word in customized_dict.keys():
        word = customized_dict[word]
    return word

In [7]:
# get the eneties for the whole data:text_df
nlp = spacy.load("en_core_web_sm")
def getEntity(text_df):
    entites_list = []
    for text in text_df:
        doc = nlp(text)
        temp_list = []
        #remove number from the entity list 
        for entity in doc.ents:
            temp = re.sub('[^a-zA-Z ]+', '', entity.text).strip()
            if temp != '':
                temp_list.append(cusomizedNameEntity(temp))
        entites_list.append(temp_list)
    return entites_list

In [8]:
#read data
df = readArticle('../data/raw/articles.txt')

In [13]:
#normalize data
text_df = df['text'].apply(clean_text)
text_df.shape

(213605,)

In [ ]:
#get entities
entites_list = getEntity(text_df)

In [ ]:
len(entites_list)

In [ ]:
with open('../data/embedding/entites.txt', 'w') as f:
    for item in entites_list:
        f.write("%s\n" % item)